In [1]:
import pandas as pd
import numpy as np
from timedelta import Timedelta 

from lumibot.brokers import Alpaca
from lumibot.backtesting import BacktestingBroker, YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from lumibot.entities import Asset
from datetime import datetime 
from alpaca_trade_api import REST 

from src.get_data import GetUpdateData
from ta import add_all_ta_features

from joblib import dump, load

In [17]:
clf_model = load(
    "models/candidate_models/lgbm-direction-classifier-2024-05-03-22-30-17.joblib"
)
reg_model = load(
    "models/candidate_models/lgbm-percentchange-regressor-2024-05-03-22-33-08.joblib"
)


class MyStrategy(Strategy):
    def initialize(self, symbol: str = "SPY", cash_at_risk: float = 1):
        self.symbol = symbol
        self.sleeptime = "7D"
        self.last_trade = None
        # self.set_market("24/7")
        self.cash_at_risk = cash_at_risk
        self.df_ohlci = self.get_data()

    def get_data(self):

        ticker = self.symbol.replace("-", "/")
        df = (
            GetUpdateData(api_key="96c18c45dfbf4554975a6c0474c297e4")
            .get_data_single_ticker(ticker)
            .sort_values("datetime", ascending=True)
        )
        df_ohlci = add_all_ta_features(
            df,
            open="open",
            high="high",
            low="low",
            close="close",
            volume="volume",
            fillna=True,
        )
        
        print(df_ohlci['datetime'].min())

        return df_ohlci

        
    
    def get_date(self): 
        today = self.get_datetime()
        return today.strftime('%Y-%m-%d')

    def get_signal(self):
        X = self.df_ohlci.copy()
        today = self.get_date()
        X = X.reset_index(drop=True)
        prev_index = X[X['datetime'] == today].index[0] - 1
        X = X.iloc[[prev_index]]
        X.drop('datetime', axis=1, inplace=True)

        pred_direction_proba = clf_model.predict_proba(X)[0][1]
        pred_percent_change = reg_model.predict(X)[0]
        
        # print(f'today_date: {today}, X Empty?: {X.empty}')

        return pred_direction_proba, pred_percent_change, X

    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price, 0)
        return cash, last_price, quantity

    def on_trading_iteration(self):

        cash, last_price, quantity = self.position_sizing()
        pred_direction_proba, pred_percent_change, X = self.get_signal()
        yesterday_close_price = X['close'].values[0]
        
        print(f'pred_direction_proba: {pred_direction_proba}, pred_percent_change: {pred_percent_change}')

        if (cash > last_price) & (pred_direction_proba >= 0.51):
            # if  pred_percent_change >= 0.001: #(last_price < (yesterday_close_price * (pred_percent_change * 100))):
            if self.last_trade == "sell":
                self.sell_all()
            order = self.create_order(
                self.symbol,
                quantity,
                "buy",
                type="bracket",
                take_profit_price=last_price * 1.10,
                stop_loss_price=last_price * 0.95,
            )
            self.submit_order(order)
            self.last_trade = "buy"
            # elif pred_direction == 1 and pred_percent_change <= 0.01:
            #     self.sell_all()
            # elif pred_direction == 0 and pred_percent_change >= -0.01:
            #     self.sell_all()
        if pred_direction_proba <= 0.45: #(last_price >= (yesterday_close_price * (pred_percent_change * 100))):
            if self.last_trade == "buy":
                self.sell_all()
                cash, last_price, quantity = self.position_sizing()
                if (cash > last_price):
                    order = self.create_order(
                        self.symbol,
                        quantity,
                        "sell",
                        type="bracket",
                        take_profit_price=last_price * 0.90,
                        stop_loss_price=last_price * 1.05,
                    )
                    self.submit_order(order)
                    self.last_trade = "sell"


# Pick the dates that you want to start and end your backtest
# and the allocated budget
start_date = datetime(2005, 1, 1)
end_date = datetime(2024, 4, 30)
# broker = Alpaca(ALPACA_CREDS)

# Run the backtest
trader = Trader(backtest=True)
data_source = YahooDataBacktesting(
    datetime_start=start_date,
    datetime_end=end_date,
)
broker = BacktestingBroker(data_source)
strat = MyStrategy(
    broker=broker,
)
trader.add_strategy(strat)
trader.run_all()

2004-06-23
Progress ||   0.03%  [Elapsed: 0:00:03 ETA: 3:12:58] pred_direction_proba: 0.5043090842395809, pred_percent_change: 0.014764780227884376
Progress ||   0.13%  [Elapsed: 0:00:04 ETA: 0:53:53] Portfolio Val: 100,000.00pred_direction_proba: 0.501061076824263, pred_percent_change: 0.007625411333444333
Progress ||   0.25%  [Elapsed: 0:00:04 ETA: 0:29:20] Portfolio Val: 100,000.00pred_direction_proba: 0.5120212971808101, pred_percent_change: 0.007752039563203073
Progress ||   0.35%  [Elapsed: 0:00:04 ETA: 0:21:04] Portfolio Val: 100,000.00pred_direction_proba: 0.5287512683663087, pred_percent_change: 0.011202675976865922
Progress ||   0.44%  [Elapsed: 0:00:04 ETA: 0:16:27] Portfolio Val: 99,034.42pred_direction_proba: 0.4755723441983241, pred_percent_change: 0.008506366722054045
Progress ||   0.54%  [Elapsed: 0:00:04 ETA: 0:13:32] Portfolio Val: 100,169.40pred_direction_proba: 0.5089709782603431, pred_percent_change: 0.007257029619626483
Progress ||   0.64%  [Elapsed: 0:00:04 ETA: 

{'MyStrategy': {'cagr': 0.12546336866457697,
  'volatility': 0.1703839572561698,
  'sharpe': 0.4289333965517104,
  'max_drawdown': {'drawdown': 0.47813110695955957,
   'date': Timestamp('2009-03-10 10:30:00-0400', tz='America/New_York')},
  'romad': 0.26240369396251956,
  'total_return': 8.793337701202406}}